In [ ]:
import collections
import pandas as pd
import melee
from slippi_db import upload_lib, test_peppi, preprocessing

In [ ]:
singles_infos = test_peppi.get_singles_info('prod')

In [ ]:
print(len(singles_infos))

In [ ]:
def to_row(game: dict) -> dict:
    row = game.copy()
    del row['players']
    for i, player in enumerate(game['players']):
        for k, v in player.items():
            row[f'p{i}.{k}'] = v
    return row

In [ ]:
df = pd.DataFrame(map(to_row, singles_infos))

In [ ]:
df.keys()

In [ ]:
def num_games_as(character: melee.Character) -> int:
    total = 0
    for i in [0, 1]:
        total += (df[f'p{i}.character'] == character.value).sum()
    return total

In [ ]:
num_games_as(melee.Character.FOX)

In [ ]:
df.to_parquet('data/prod.pq')

In [ ]:
objects = upload_lib.bucket.objects.filter(
    Prefix='prod/pq/',
    MaxKeys=1_000_000,
)
objects = list(objects)

In [ ]:
len(objects)

In [ ]:
s3_keys = []
for obj in objects:
    key = obj.key.split('/')[-1]
    s3_keys.append(key)
s3_keys = set(s3_keys)

In [ ]:
parsed_db = upload_lib.get_db('prod', upload_lib.PQ)
parse_failures = list(parsed_db.find({'failed': True}))

In [ ]:
failed_keys = set(doc['key'] for doc in parse_failures)

In [ ]:
# check that all but failed parses are in s3 bucket
for key in df['key']:
    if key in s3_keys:
        continue
    
    assert key in failed_keys